# Investigating Lift to understand Problem Feasibility


1. Randomly pick customer $\vec{x}$, $\text{Prob}(\text{Churn})=0.2$ Using ML identify and rank churning customers
2. Sort states based on Prob
3. In our cases feasible states should be front-loaded
3. Did it bring forward the chance of finding the feasibility
4. How do we prioritise feasibility vs optimality
5. Y axis represents a measure of how many feasible states have been explored (by the sorted list of probabilties).
6. Ideally want to see a front loading in the lift
7. What happens when we turn the cost on?
8. Identify AREA under the curve

How much does it care about cost?

In [1]:
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Standard Libraries
import argparse
import json
import time
import networkx as nx
import numpy as np
from joblib import Parallel, delayed
import mlflow
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pylab

# Custom Libraries
import qaoa_vrp.build_graph
import qaoa_vrp.features.graph_features
import qaoa_vrp.features.tsp_features
import qaoa_vrp.build_circuit
import qaoa_vrp.clustering
import qaoa_vrp.utils
from qaoa_vrp.exp_utils import str2bool, make_temp_directory
from qaoa_vrp.quantum_burden import compute_quantum_burden
from qaoa_vrp.classical.greedy_tsp import greedy_tsp
from qaoa_vrp.plot.draw_euclidean_graphs import draw_euclidean_graph

# QISKIT stuff
from qiskit import Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver,VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NELDER_MEAD, SPSA
from qiskit.circuit import Parameter
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
)
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.applications.ising import max_cut, tsp

# import logging
# logging.basicConfig(level=logging.INFO)
# logging.getLogger('qiskit.algorithms.minimum_eigen_solvers.vqe').setLevel(logging.INFO)G = # Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Standard Libraries
import argparse
import json
import time
import networkx as nx
import numpy as np
from joblib import Parallel, delayed
import mlflow
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pylab

# Custom Libraries
import qaoa_vrp.build_graph
import qaoa_vrp.features.graph_features
import qaoa_vrp.features.tsp_features
import qaoa_vrp.build_circuit
import qaoa_vrp.clustering
import qaoa_vrp.utils
from qaoa_vrp.exp_utils import str2bool, make_temp_directory
from qaoa_vrp.quantum_burden import compute_quantum_burden
from qaoa_vrp.classical.greedy_tsp import greedy_tsp
from qaoa_vrp.plot.draw_euclidean_graphs import draw_euclidean_graph
from qaoa_vrp.parallel.optimize_qaoa import run_qaoa_parallel

# QISKIT stuff
from qiskit import Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver,VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NELDER_MEAD, SPSA
from qiskit.circuit import Parameter
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
)
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.optimization.applications.ising import max_cut, tsp

# import logging
# logging.basicConfig(level=logging.INFO)
# logging.getLogger('qiskit.algorithms.minimum_eigen_solvers.vqe').setLevel(logging.INFO)

/Users/vivekkatial/.local/share/virtualenvs/HAQC-azaY3fAi/lib/python3.9/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/Users/vivekkatial/.local/share/virtualenvs/HAQC-azaY3fAi/lib/python3.9/site-packages/qiskit/finance/__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')
/Users/vivekkatial/.local/share/virtualenvs/HAQC-azaY3fAi/lib/python3.9/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimizati

In [4]:
filename="instanceType_euclidean_tsp_numNodes_4_numVehicles_1_87a170c748e240d0b71d5fb7fe7de707.json"
instance_path = "../data/{}".format(filename)
with open(instance_path) as f:
        data = json.load(f)
        
# Build Graph
G, depot_info = qaoa_vrp.build_graph.build_json_graph(data["graph"])
num_vehicles = int(data["numVehicles"])
threshold = float(data["threshold"])
n_max = int(data["n_max"])
instance_type = data["instance_type"]
p_max = data["p_max"]
depot_edges = list(G.edges(depot_info["id"], data=True))
depot_node = depot_info["id"]
edge_mat = nx.linalg.graphmatrix.adjacency_matrix(G).toarray()
cost_mat = np.array(nx.attr_matrix(G, edge_attr="cost", rc_order=list(G.nodes())))
for edge in G.edges():
    G[edge[0]][edge[1]]['cost'] = 0
edge_mat = nx.linalg.graphmatrix.adjacency_matrix(G).toarray()
cost_mat = np.array(nx.attr_matrix(G, edge_attr="cost", rc_order=list(G.nodes())))

G, cluster_mapping = qaoa_vrp.clustering.create_clusters(
    G, num_vehicles, "spectral-clustering", edge_mat
)

# Extract subgraph
subgraphs = qaoa_vrp.clustering.build_sub_graphs(G, depot_node, depot_edges)

# big_offset = sum(sum(cost_mat))/2 + 1
big_offset=30
qubos = qaoa_vrp.build_circuit.build_qubos(subgraphs, depot_info,A=big_offset)
cluster_mapping = [i + 1 for i in cluster_mapping]
cluster_mapping.insert(0, 0)
qubo = qubos[0]
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 120 X11 - 120 X12 - 120 X13 - 120 X21 - 120 X22 - 120 X23 - 120 X31
      - 120 X32 - 120 X33 + [ 120 X11^2 + 120 X11*X12 + 120 X11*X13
      + 120 X11*X21 + 120 X11*X31 + 120 X12^2 + 120 X12*X13 + 120 X12*X22
      + 120 X12*X32 + 120 X13^2 + 120 X13*X23 + 120 X13*X33 + 120 X21^2
      + 120 X21*X22 + 120 X21*X23 + 120 X21*X31 + 120 X22^2 + 120 X22*X23
      + 120 X22*X32 + 120 X23^2 + 120 X23*X33 + 120 X31^2 + 120 X31*X32
      + 120 X31*X33 + 120 X32^2 + 120 X32*X33 + 120 X33^2 ]/2 + 180
Subject To

Bounds
 0 <= X11 <= 1
 0 <= X12 <= 1
 0 <= X13 <= 1
 0 <= X21 <= 1
 0 <= X22 <= 1
 0 <= X23 <= 1
 0 <= X31 <= 1
 0 <= X32 <= 1
 0 <= X33 <= 1

Binaries
 X11 X12 X13 X21 X22 X23 X31 X32 X33
End

In [5]:
op, offset = qubo.to_ising()
num_qubits = qubo.get_num_vars()
print('Offset:', offset)
print('Ising Hamiltonian:')
print(op.print_details())

Offset: 180.0
Ising Hamiltonian:
IIIIIIIIZ	(-30+0j)
IIIIIIIZI	(-30+0j)
IIIIIIZII	(-30+0j)
IIIIIZIII	(-30+0j)
IIIIZIIII	(-30+0j)
IIIZIIIII	(-30+0j)
IIZIIIIII	(-30+0j)
IZIIIIIII	(-30+0j)
ZIIIIIIII	(-30+0j)
IIIIIIIZZ	(15+0j)
IIIIIIZIZ	(15+0j)
IIIIIIZZI	(15+0j)
IIIIIZIIZ	(15+0j)
IIIIZIIZI	(15+0j)
IIIIZZIII	(15+0j)
IIIZIIZII	(15+0j)
IIIZIZIII	(15+0j)
IIIZZIIII	(15+0j)
IIZIIIIIZ	(15+0j)
IIZIIZIII	(15+0j)
IZIIIIIZI	(15+0j)
IZIIZIIII	(15+0j)
IZZIIIIII	(15+0j)
ZIIIIIZII	(15+0j)
ZIIZIIIII	(15+0j)
ZIZIIIIII	(15+0j)
ZZIIIIIII	(15+0j)



In [6]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
qp.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: AnonymousModel
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool x_5;
dvar bool x_6;
dvar bool x_7;
dvar bool x_8;

minimize
 - 60 x_0 - 60 x_1 - 60 x_2 - 60 x_3 - 60 x_4 - 60 x_5 - 60 x_6 - 60 x_7
 - 60 x_8 [ 60 x_0*x_1 + 60 x_0*x_2 + 60 x_0*x_3 + 60 x_0*x_6 + 60 x_1*x_2
 + 60 x_1*x_4 + 60 x_1*x_7 + 60 x_2*x_5 + 60 x_2*x_8 + 60 x_3*x_4 + 60 x_3*x_5
 + 60 x_3*x_6 + 60 x_4*x_5 + 60 x_4*x_7 + 60 x_5*x_8 + 60 x_6*x_7 + 60 x_6*x_8
 + 60 x_7*x_8 ] + 180;
 
subject to {

}


In [7]:
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact.solve(qp)
print(exact_result)

optimal function value: 0.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS


In [9]:
ee = NumPyMinimumEigensolver(op)
exact_result = ee.run()
print('energy:', exact_result.eigenvalue.real)
print('tsp objective:', exact_result.eigenvalue.real + offset)
x = sample_most_likely(exact_result.eigenstate)
print('feasible:', tsp.tsp_feasible(x))
z = tsp.get_tsp_solution(x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, cost_mat))

energy: -180.0
tsp objective: 0.0
feasible: True
solution: [1, 2, 0]
solution objective: 0.0


In [10]:
budget=100
optimizers = [
    COBYLA(maxiter=budget, disp=True, rhobeg=0.1)
]
converge_cnts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

In [15]:
result = run_qaoa_parallel(args=[optimizers[0], budget,op,4, False])

 Running Optimizer: COBYLA in parallel

   Normal return from subroutine COBYLA

   NFVALS =   76   F =-4.951172E+01    MAXCV = 0.000000E+00
   X = 7.303042E-02   3.660271E-02   3.699958E-01   7.432987E-01   7.220899E-01
       7.572131E-01   4.427198E-01   2.050239E-01

   Normal return from subroutine COBYLA

   NFVALS =   95   F =-1.248633E+02    MAXCV = 0.000000E+00
   X = 5.266339E-01   8.610512E-01   5.285981E-01   4.062496E-01   4.691383E-01
       6.945307E-01   5.213929E-01   9.364206E-01
 Ending run for  Optimizer: COBYLA in parallel


1. In our cases feasible states should be front-loaded
2. Did it bring forward the chance of finding the feasibility
3. How do we prioritise feasibility vs optimality
4. Y axis represents a measure of how many feasible states have been explored (by the sorted list of probabilties).
5. Ideally want to see a front loading in the lift
6. What happens when we turn the cost on?
7. Identify AREA under the curve

In [16]:
result["min_energy_state"]

{'101111111': 1,
 '101111000': 1,
 '001111000': 1,
 '000111111': 1,
 '100010111': 1,
 '110110001': 1,
 '101011001': 1,
 '100111110': 1,
 '100100110': 1,
 '101101100': 1,
 '011110110': 1,
 '100011001': 2,
 '010011010': 1,
 '101100000': 1,
 '010011100': 3,
 '010001001': 8,
 '010111010': 1,
 '001000111': 2,
 '111100010': 1,
 '100101100': 1,
 '110001010': 1,
 '011011101': 1,
 '100101010': 1,
 '101000111': 1,
 '110111000': 1,
 '111001100': 1,
 '001010111': 1,
 '001110111': 1,
 '111000011': 1,
 '001101011': 1,
 '010110010': 1,
 '101000010': 4,
 '100010011': 1,
 '001111111': 1,
 '010100100': 7,
 '100101000': 3,
 '100001011': 1,
 '011000001': 1,
 '110110101': 1,
 '010000111': 2,
 '000100110': 1,
 '001011100': 2,
 '001101010': 2,
 '100111000': 2,
 '100110001': 3,
 '000001100': 5,
 '001000100': 2,
 '110111111': 1,
 '111000111': 1,
 '000010111': 1,
 '000001101': 2,
 '100001001': 3,
 '001100110': 4,
 '010011110': 1,
 '110100101': 1,
 '011000111': 2,
 '001010000': 1,
 '001001010': 5,
 '010101001': 